# Definiendo una geogranja

En esta libreta crearemos una aplicacion que defina una region geografica y veremos como reaccionar ante eventos de localizacion: cuando el dispositivo salga de la region definida, la geogranja, se emite un mensaje y se registra que se salio de la region. Cuando entramos nuevamente a la region se notifica y se registra la entrada.

In [7]:
# import android
import android

In [93]:
!adb devices

List of devices attached
09c3400a	device



In [94]:
!adb forward tcp:9999 tcp:2222

In [95]:
droid = android.Android() #["192.168.100.74","2222"])
droid.ttsSpeak("listo para recibir instrucciones")

Result(id=0, result=None, error=None)

In [102]:
lat1 = 19.5197093
lon1 = -96.9155696
hd = 0
prov = '-'
newLoc = False
curTime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
"{0}, {1}, {2}, {3}, {4}, {5}\n".format(curTime, lat1, lon1, hd, prov, newLoc)

'2018-03-23 15:24:48, 19.5197093, -96.9155696, 0, -, False\n'

In [103]:
%%file granja.py
import android
import datetime, time
from haversine import *
droid = android.Android()

logName = 'locationLog-'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
localLog = '/sdcard/sl4a/scripts/RySI/'+logName+'.txt'

logHeader = "Time, Latitude, Longitude, Distance, Provider\n"

lat1 = 19.5197093
lon1 = -96.9155696
hd = haversine(lat1, lon1, lat1, lon1)
prov = '-'

droid.wakeLockAcquirePartial()
droid.startLocating()
time.sleep(15)

with open(localLog, 'a') as logFile:
    logFile.write(logHeader)
    curTime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    logFile.write("{0}, {1}, {2}, {3}, {4}, {5}\n".format(curTime, lat1, lon1, hd, prov, '-'))
    
    while True:
        loc = droid.readLocation().result
        if loc == {}:
            loc = droid.getLastKnownLocation().result
            newLoc = False
        if loc != {}:
            newLoc = True
            try:
                prov = 'gps'
                n = loc[prov]                
            except KeyError:
                prov = 'network'
                n = loc[prov]
            
        la = n['latitude']
        lo = n['longitude']
        ts = n['time']
        t = datetime.datetime.fromtimestamp(ts/1e3).strftime('%H:%M:%S')
        
        hd = haversine(la, lo, lat1, lon1)
        time.sleep(10)
        
        logLine = "{0}, {1}, {2}, {3}, {4}, {5}\n".format(t, lat1, lon1, hd, prov, newLoc)
        logFile.write(logLine)
        logFile.flush()
        
        print "{0}, {1}, {2}, {3}".format(t, hd, prov, newLoc)
            
        if hd > .1:
            droid.toggleRingerSilentMode(True)
            droid.vibrate(1000)
            droid.ttsSpeak("Dejando geocerca")
        else:
            droid.toggleRingerSilentMode(False)
    
    logFile.close()


Overwriting granja.py


In [27]:
%%file haversine.py
# haversine function to get distance

from math import * 
 
def haversine(lon1, lat1, lon2, lat2):
    """     
    Calculate the great circle distance between two points      
    on the earth (specified in decimal degrees)     
    """     
    # convert decimal degrees to radians      
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])     
    # haversine formula      
    dlon = lon2 - lon1      
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2     
    c = 2 * atan2(sqrt(a), sqrt(1-a))      
    km = 6367 * c    
    return km  

Overwriting haversine.py


In [32]:
!adb push haversine.py /sdcard/sl4a/scripts/RySI

13 KB/s (596 bytes in 0.042s)


In [104]:
!adb push granja.py /sdcard/sl4a/scripts/RySI

40 KB/s (1777 bytes in 0.042s)
